In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import bs4
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
#As the application getting complex, you should be able to inspect what exactly going on inside your chain or agents
os.environ["LANGCHAIN_TRACING_V2"] = "true"


llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
#Loading
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-header","post-content")
        )
    )
)
docs= loader.load()
chunks = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200).split_documents(docs)



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
from langchain_core.prompts import ChatPromptTemplate
#Store Embeddings into Vector DB
embeddings = OpenAIEmbeddings()
vectorDB= Chroma.from_documents(chunks,embeddings)
#create a retriever for this DB
retriever = vectorDB.as_retriever()
# create a system prompt
system_prompt= " You are acting as an assistant to Question-Answering Tasks. Use the context given in triple backticks to answer the question.```{context}``` If you dont know answer jut say Dont know. Keep the answer as short using only 4 sentences  "
#prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
stuff_chain = create_stuff_documents_chain(llm,prompt)
# response = create_retrieval_chain(retriever,stuff_chain)
# response["answer"]
#Upto here, It doesn't store your previous conversation what you had.
#below we have implemented in such a way that it stores your previous chat and understands the context

context_based_system_prompt =(
        "Latest question and Chat History has been given"
        "This latest questin might reference the context in the chat history"
        "Formuate a standalone question in a undestandable way with out the chat history"
        "Do not Answer the question just reformulate it if needed, else, return as it is"
        )
context_based_prompt = ChatPromptTemplate.from_messages(
   [
        ("system",context_based_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
   ] 
    
)

In [11]:
history_retriever = create_history_aware_retriever(llm,retriever,context_based_prompt)
chat_qa_prompt=ChatPromptTemplate.from_messages(
        [
            ("system",system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human","{input}"),
         ]
    
)
stuff_qa_chain = create_stuff_documents_chain(llm,chat_qa_prompt)

final_rag_chain = create_retrieval_chain(history_retriever,stuff_qa_chain)


In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage,HumanMessage

store={}
#to get session
def get_SessionHistory(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [20]:
stateful_rag_chain = RunnableWithMessageHistory(
    final_rag_chain,
    get_SessionHistory,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"

)

In [21]:
response =stateful_rag_chain.invoke(
    {"input":"What is Task Decompossion?"},
    config={
        "configurable":{"session_id":"xyz123"}
    },

)
response["answer"]

Parent run 314075b6-9fca-40ad-9bcf-0b1e392592a7 not found for run 995de541-5849-451e-bea3-f1b912cf3db7. Treating as a root run.


"Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps models tackle hard tasks by transforming them into more manageable subtasks. It can be implemented through simple prompts for language models, task-specific instructions, or human inputs to guide the model's thinking process."

In [22]:
response =stateful_rag_chain.invoke(
    {"input":"What are common ways of doing it?"},
    config={
        "configurable":{"session_id":"xyz123"}
    },

)
response["answer"]

Parent run de07e289-413c-405a-ac9c-7ac4b27178fb not found for run dca52ea0-8539-4f2a-8d8d-17036c19dbb2. Treating as a root run.


'Task decomposition can be achieved through simple prompting for language models, such as instructing them to think step by step or providing subgoals for a specific task. Task-specific instructions tailored to the nature of the task, like writing a story outline for novel writing, can also guide the decomposition process. Additionally, human inputs can be used to break down complex tasks into smaller steps for the model to follow.'

In [25]:
for message in store["xyz123"].messages:
    if isinstance(message, AIMessage):
        prefix = "Assistant"
    else:
        prefix = "You"

    print(f"{prefix}: {message.content}\n")

You: What is Task Decompossion?

Assistant: Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps models tackle hard tasks by transforming them into more manageable subtasks. It can be implemented through simple prompts for language models, task-specific instructions, or human inputs to guide the model's thinking process.

You: What are common ways of doing it?

Assistant: Task decomposition can be achieved through simple prompting for language models, such as instructing them to think step by step or providing subgoals for a specific task. Task-specific instructions tailored to the nature of the task, like writing a story outline for novel writing, can also guide the decomposition process. Additionally, human inputs can be used to break down complex tasks into smaller steps for the model to follow.

